In [99]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [100]:
data = pd.read_json('acndata_sessions.json')


In [109]:
# print(data['_items'])

list = data['_items'].to_list()

df = pd.DataFrame(list)
display(df)

,_id,clusterID,connectionTime,disconnectTime,doneChargingTime,kWhDelivered,sessionID,siteID,spaceID,stationID,timezone,userID,userInputs
0,5c412c1df9af8b12cb56c27c,0039,"Tue, 01 Jan 2019 17:41:45 GMT","Tue, 01 Jan 2019 18:39:21 GMT","Tue, 01 Jan 2019 18:40:21 GMT",0.900,2_39_138_566_2019-01-01 17:41:44.784919,0002,CA-512,2-39-138-566,America/Los_Angeles,None,None
1,5c412c1df9af8b12cb56c27d,0039,"Tue, 01 Jan 2019 18:09:17 GMT","Wed, 02 Jan 2019 02:39:32 GMT","Tue, 01 Jan 2019 20:16:10 GMT",12.534,2_39_79_379_2019-01-01 18:09:16.991864,0002,CA-327,2-39-79-379,America/Los_Angeles,000000558,"[{'WhPerMile': 222, 'kWhRequested': 17.76, 'mi..."
2,5c412c1df9af8b12cb56c27e,0039,"Tue, 01 Jan 2019 18:39:25 GMT","Tue, 01 Jan 2019 19:18:49 GMT","Tue, 01 Jan 2019 19:19:47 GMT",0.883,2_39_138_566_2019-01-01 18:39:24.566872,0002,CA-512,2-39-138-566,America/Los_Angeles,None,None
3,5c412c1df9af8b12cb56c27f,0039,"Tue, 01 Jan 2019 19:18:53 GMT","Tue, 01 Jan 2019 20:14:07 GMT","Tue, 01 Jan 2019 19:59:08 GMT",0.879,2_39_138_566_2019-01-01 19:18:52.843645,0002,CA-512,2-39-138-566,America/Los_Angeles,None,None
4,5c412c1df9af8b12cb56c280,0039,"Tue, 01 Jan 2019 21:05:57 GMT","Wed, 02 Jan 2019 02:03:02 GMT","Wed, 02 Jan 2019 01:59:27 GMT",16.136,2_39_79_378_2019-01-01 21:05:56.972890,0002,CA-326,2-39-79-378,America/Los_Angeles,000001135,"[{'WhPerMile': 400, 'kWhRequested': 20.0, 'mil..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
16120,61550519f9af8b76960e169a,0039,"Mon, 13 Sep 2021 21:12:53 GMT","Tue, 14 Sep 2021 00:25:36 GMT","Mon, 13 Sep 2021 21:41:31 GMT",2.285,2_39_89_25_2021-09-13 21:12:53.318460,0002,CA-315,2-39-89-25,America/Los_Angeles,000000431,"[{'WhPerMile': 286, 'kWhRequested': 28.6, 'mil..."
16121,61550519f9af8b76960e169b,0039,"Mon, 13 Sep 2021 21:17:04 GMT","Tue, 14 Sep 2021 01:01:49 GMT","Mon, 13 Sep 2021 23:18:07 GMT",6.715,2_39_123_23_2021-09-13 21:16:44.026068,0002,CA-313,2-39-123-23,America/Los_Angeles,000006481,"[{'WhPerMile': 460, 'kWhRequested': 11.5, 'mil..."
16122,61550519f9af8b76960e169c,0039,"Mon, 13 Sep 2021 22:33:07 GMT","Mon, 13 Sep 2021 23:06:55 GMT",None,17.720,2_39_81_4550_2021-09-13 22:33:04.543952,0002,11900388,2-39-81-4550,America/Los_Angeles,000003905,"[{'WhPerMile': 300, 'kWhRequested': 30.0, 'mil..."
16123,61550519f9af8b76960e169d,0039,"Mon, 13 Sep 2021 23:11:12 GMT","Tue, 14 Sep 2021 01:43:11 GMT","Tue, 14 Sep 2021 00:13:35 GMT",2.018,2_39_91_437_2021-09-13 23:10:59.528292,0002,CA-317,2-39-91-437,America/Los_Angeles,000001082,"[{'WhPerMile': 290, 'kWhRequested': 5.8, 'mile..."


In [110]:
df["connectionTime"] = pd.to_datetime(df["connectionTime"])
df["disconnectTime"] = pd.to_datetime(df["disconnectTime"])

# round each time to the last hour
df["connectionHour"] = df["connectionTime"].dt.floor("H")

# round each time to the next hour
df["disconnectHour"] = df["disconnectTime"].dt.ceil("H")

# compute duration of charging sessions
df["duration"] = (df["disconnectHour"] - df["connectionHour"]).dt.seconds / 3600

# drop unnecessary columns
df.drop(["connectionTime", "disconnectTime", "sessionID", "clusterID", "siteID", "userID", "userInputs"], axis=1, inplace=True)

display(df)

print(len(df))

print(len(df[df["duration"] > 6]))

,_id,doneChargingTime,kWhDelivered,spaceID,stationID,timezone,connectionHour,disconnectHour,duration
0,5c412c1df9af8b12cb56c27c,"Tue, 01 Jan 2019 18:40:21 GMT",0.900,CA-512,2-39-138-566,America/Los_Angeles,2019-01-01 17:00:00+00:00,2019-01-01 19:00:00+00:00,2.0
1,5c412c1df9af8b12cb56c27d,"Tue, 01 Jan 2019 20:16:10 GMT",12.534,CA-327,2-39-79-379,America/Los_Angeles,2019-01-01 18:00:00+00:00,2019-01-02 03:00:00+00:00,9.0
2,5c412c1df9af8b12cb56c27e,"Tue, 01 Jan 2019 19:19:47 GMT",0.883,CA-512,2-39-138-566,America/Los_Angeles,2019-01-01 18:00:00+00:00,2019-01-01 20:00:00+00:00,2.0
3,5c412c1df9af8b12cb56c27f,"Tue, 01 Jan 2019 19:59:08 GMT",0.879,CA-512,2-39-138-566,America/Los_Angeles,2019-01-01 19:00:00+00:00,2019-01-01 21:00:00+00:00,2.0
4,5c412c1df9af8b12cb56c280,"Wed, 02 Jan 2019 01:59:27 GMT",16.136,CA-326,2-39-79-378,America/Los_Angeles,2019-01-01 21:00:00+00:00,2019-01-02 03:00:00+00:00,6.0
...,...,...,...,...,...,...,...,...,...
16120,61550519f9af8b76960e169a,"Mon, 13 Sep 2021 21:41:31 GMT",2.285,CA-315,2-39-89-25,America/Los_Angeles,2021-09-13 21:00:00+00:00,2021-09-14 01:00:00+00:00,4.0
16121,61550519f9af8b76960e169b,"Mon, 13 Sep 2021 23:18:07 GMT",6.715,CA-313,2-39-123-23,America/Los_Angeles,2021-09-13 21:00:00+00:00,2021-09-14 02:00:00+00:00,5.0
16122,61550519f9af8b76960e169c,None,17.720,11900388,2-39-81-4550,America/Los_Angeles,2021-09-13 22:00:00+00:00,2021-09-14 00:00:00+00:00,2.0
16123,61550519f9af8b76960e169d,"Tue, 14 Sep 2021 00:13:35 GMT",2.018,CA-317,2-39-91-437,America/Los_Angeles,2021-09-13 23:00:00+00:00,2021-09-14 02:00:00+00:00,3.0


16125
7233


In [111]:
# read carbon intensities from CISO csv file

cf = pd.read_csv('CISO_direct_emissions.csv', parse_dates=True)

cf['datetime'] = pd.to_datetime(cf['UTC time'], utc=True)

cf.drop(["Unnamed: 0", "UTC time"], axis=1, inplace=True)
cf.set_index('datetime', inplace=True)

# print the types in the dataframe
# print(cf.index.dtype)
# display(cf)

print(cf.loc['2019-08-20 09'])

carbon_intensity     163.3
coal                   7.0
nat_gas             6808.0
nuclear             2256.0
oil                   81.0
hydro               3143.0
solar                  0.0
wind                3364.0
other                  0.0
Name: 2019-08-20 09:00:00+00:00, dtype: float64


In [112]:
i = 2
print(df['connectionHour'][i])

print(cf.loc[df['connectionHour'][i]:df['disconnectHour'][i]])

2019-01-01 18:00:00+00:00
                           carbon_intensity  coal  nat_gas  nuclear   oil  \
datetime                                                                    
2019-01-01 18:00:00+00:00             82.63  12.0   3656.0   2273.0  90.0   
2019-01-01 19:00:00+00:00             79.85  13.0   3538.0   2274.0  92.0   
2019-01-01 20:00:00+00:00             81.81  14.0   3559.0   2272.0  89.0   

                            hydro   solar    wind  other  
datetime                                                  
2019-01-01 18:00:00+00:00  1187.0  7931.0  1774.0    0.0  
2019-01-01 19:00:00+00:00  1077.0  7909.0  2082.0    0.0  
2019-01-01 20:00:00+00:00  1064.0  7831.0  1839.0    0.0  


In [113]:
# generate sequences:
import pyximport
pyximport.install()
import functions as f

df = df[df['duration'] > 5]
print(len(df))
print(df['duration'].mean())

df = df[df['kWhDelivered'] < 18]
print(len(df))
print(df['duration'].mean())
display(df)

# j = 0

# # for each charging session, (i.e. row in the dataframe)
# for i, row in enumerate(df.itertuples()):
#     connect = row.connectionHour
#     disconnect = row.disconnectHour
#     carbon = cf.loc[connect:disconnect]
#     seq = carbon['carbon_intensity'].tolist() # get the ground truth carbon intensity for that location

#     # set the switching cost
#     beta = 20

#     # get the optimal solution
#     opt, optCost = f.optimalSolution(seq, beta)

#     # print the optimal solution
#     print("OPT: ", opt[0:len(seq)], " OPT Cost: ", optCost)

#     j += 1
#     if j > 10:
#         break

8201
9.237532008291671
6973
9.157464505951527


,_id,doneChargingTime,kWhDelivered,spaceID,stationID,timezone,connectionHour,disconnectHour,duration
1,5c412c1df9af8b12cb56c27d,"Tue, 01 Jan 2019 20:16:10 GMT",12.534,CA-327,2-39-79-379,America/Los_Angeles,2019-01-01 18:00:00+00:00,2019-01-02 03:00:00+00:00,9.0
4,5c412c1df9af8b12cb56c280,"Wed, 02 Jan 2019 01:59:27 GMT",16.136,CA-326,2-39-79-378,America/Los_Angeles,2019-01-01 21:00:00+00:00,2019-01-02 03:00:00+00:00,6.0
6,5c412c1df9af8b12cb56c282,"Wed, 02 Jan 2019 01:18:33 GMT",1.280,CA-325,2-39-79-377,America/Los_Angeles,2019-01-01 21:00:00+00:00,2019-01-02 03:00:00+00:00,6.0
9,5c427db1f9af8b1e1a538288,"Wed, 02 Jan 2019 19:05:25 GMT",8.855,CA-491,2-39-79-382,America/Los_Angeles,2019-01-02 15:00:00+00:00,2019-01-03 01:00:00+00:00,10.0
10,5c427db1f9af8b1e1a538289,"Wed, 02 Jan 2019 19:10:32 GMT",10.402,CA-493,2-39-78-361,America/Los_Angeles,2019-01-02 16:00:00+00:00,2019-01-03 02:00:00+00:00,10.0
...,...,...,...,...,...,...,...,...,...
16102,61550519f9af8b76960e1688,"Mon, 13 Sep 2021 19:55:46 GMT",15.495,CA-309,2-39-127-19,America/Los_Angeles,2021-09-13 15:00:00+00:00,2021-09-14 01:00:00+00:00,10.0
16108,61550519f9af8b76960e168e,"Mon, 13 Sep 2021 19:19:00 GMT",4.841,CA-324,2-39-79-376,America/Los_Angeles,2021-09-13 16:00:00+00:00,2021-09-14 03:00:00+00:00,11.0
16112,61550519f9af8b76960e1692,"Mon, 13 Sep 2021 19:28:06 GMT",6.149,CA-319,2-39-95-27,America/Los_Angeles,2021-09-13 17:00:00+00:00,2021-09-14 02:00:00+00:00,9.0
16116,61550519f9af8b76960e1696,"Mon, 13 Sep 2021 20:01:20 GMT",4.331,CA-304,2-39-138-29,America/Los_Angeles,2021-09-13 18:00:00+00:00,2021-09-14 00:00:00+00:00,6.0
